In [3]:
import pandas as pd

In [10]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

print(len(df.columns))

19


In [12]:
# ...existing code...
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

print(df['duration'].std())
# ...existing code...

42.594351241920904


In [14]:
initial_count = len(df)
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
filtered_count = len(df)
fraction_left = filtered_count / initial_count

print(f"Fraction of records left: {fraction_left:.3f}")

Fraction of records left: 1.000


In [16]:
from sklearn.feature_extraction import DictVectorizer

# Re-cast IDs to strings
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Create list of dictionaries
dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit DictVectorizer
dv = DictVectorizer()
X = dv.fit_transform(dicts)

# Print the dimensionality (number of columns)
print(f"Feature matrix shape: {X.shape}")

Feature matrix shape: (3009173, 515)


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Target variable
y = df['duration'].values

# Train linear regression
lr = LinearRegression()
lr.fit(X, y)

# Predict on training data
y_pred = lr.predict(X)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y, y_pred))
print(f"RMSE on train: {rmse:.2f}")

RMSE on train: 7.65


In [21]:
df_2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [27]:
# ...existing code...
df_2['duration'] = (df_2['tpep_dropoff_datetime'] - df_2['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter out outliers in validation set
df_2 = df_2[(df_2['duration'] >= 1) & (df_2['duration'] <= 60)]

dicts_val = df_2[['PULocationID', 'DOLocationID']].to_dict(orient='records')
X_val = dv.transform(dicts_val)
y_val = df_2['duration'].values

# Predict on validation data
y_pred = lr.predict(X_val)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE on validation: {rmse:.2f}")

RMSE on validation: 7.81
